# Migros API to extract Food Data

This is for extracting the data from the Migros API to get food/logistic data. There will be several categories of this speicified later on.

In [ ]:
hackzurich2020
uhSyJ08KexKn4ZFS

In [7]:
import requests
import json

In [94]:
#Products API, with verbosity = id and limit=2000, then go through the page ids of all products
with open('ingredients_de.txt', 'r', encoding='utf-8') as file:
    ingredients_de = file.readlines()

    
ingredients_de = [x.strip() for x in ingredients_de] 

print(ingredients_de) 
recipes = []
for ingredient in ingredients_de:
    print(ingredient)
    response_code = 100
    size = 100
    
    while(response_code != 200):
        url = f"https://hackzurich-api.migros.ch/hack/recipe/recipes_de/_search?q={ingredient}&size={size}"
        response = requests.get(url, auth=('hackzurich2020', 'uhSyJ08KexKn4ZFS'))
        if(size == 20):
            size = 0
            break
        if(response.status_code != 200):
            size = 20
            print(ingredient)
        response_code = response.status_code
    if(size != 0):
        response_json = response.json()
        recipe = {}
        for hit in response_json['hits']['hits']:
            recipe['id'] = hit['_id']
            recipe['title'] = hit['_source']['title']
            for taxonomy in hit['_source']['taxonomies']:
                if taxonomy['name'] == 'Vegetarisch':
                    recipe['Veg'] = True
                elif taxonomy['name'] == 'Vegan':
                    recipe['Vegan'] = True
            if 'Veg' not in recipe:
                recipe['Veg'] = False
                recipe['Vegan'] = False
            elif 'Vegan' not in recipe:
                recipe['Veg'] = False
            recipe['url'] = hit['_source']['urls'][0]['url']

            size = hit['_source']['sizes'][0]

            quanity = size['quantity']
            ingredients = size['ingredient_blocks'][0]['ingredients']
            recipe['ingredients'] = ingredients
            recipes.append(recipe)
    #For each ingredient, what do we want to keep the recipes found for 2 person
    


['Gurke', 'Frühlingszwiebel', 'Milch', 'Haselnüsse', 'Termine', 'Karotten', 'Lauch', 'Harissa', 'maracuja', 'PASTINAKEN', 'Kidneybohnen.', 'Mango', 'Tofu', 'Thymus vulgaris', 'Zwiebeln', 'Artischocken', 'Kohlrabi', 'Pinienkerne', 'Olive', 'Zitronensaft', 'Vanille', 'Polenta', 'Parmesan', 'Senf', 'Aubergine', 'Paprika', 'Soja-Sauce', 'Lachs', 'Kirschtomaten', 'Karotte', 'Spinat', 'Joghurt', 'Koriander', 'Kokosmilch', 'Kichererbsen', 'Brokkoli', 'Eis/salat', 'orange', 'saure Sahne', 'Rindfleisch', 'Schinken', 'Reis', 'Erdbeere', 'Balsamico', 'Erdnüsse', 'Kohl', 'FETA', 'Chili', 'Chicorée', 'Kürbis', 'Pancetta', 'Sojabohnen', 'Pastinaken', 'Schwarze Bohnen', 'ROTE LINSEN', 'Banane', 'Shrimps', 'Rübe', 'Huhn', 'Chinakohl', 'Aprikose', 'die Papaya', 'Kaschu-Nüsse', 'Fisch', 'Erbsen', 'baguette', 'Wirsingkohl', 'Pflanzenöl', 'Rosmarin', 'Pilze', 'Peperoni', 'Quark', 'Ketchup', 'Sambal', 'Sukini', 'Schweinefleisch', 'Ziegenkäse', 'Birnen', 'Spaghetti', 'Trüffel', 'Limonensaft', 'Penne', 'Kalb

In [95]:
f = open("Migros_recipes.json","w")
json.dump(recipes, f)
f.close()

In [85]:
import json

In [129]:
import requests
import json
import pandas as pd
import math
import re
from requests.auth import HTTPBasicAuth

YOUR_KEY = "SR012UWDjBT7fgivtbNZSp1IJylocm5a"
AUTH = HTTPBasicAuth(YOUR_KEY, "")
BASE_URL = "https://co2.eaternity.ch"

if YOUR_KEY == "CHANGEME":
    raise RuntimeError("Please change your api key!!")

def create_kitchen(name, kitchen_id, location):
    url = f"{BASE_URL}/api/kitchens/{kitchen_id}"

    body = {
        "kitchen": {
            "name": name,
            "location": location
        }
    }

    response = requests.put(url, json=body, auth=AUTH)
    if response.status_code not in [200, 201, 202]:
        print(f"ERROR: Failed PUTting kitchen {kitchen_id} with status {response.status_code}: '{response.text}'")
    else:
        print(f"SUCCESS: PUT kitchen {kitchen_id}")
        return response.json()

    
def put_recipe_migros(recipe_id, recipe_name, kitchen_id, ingredients, bad_ingr):
    url = f"{BASE_URL}/api/kitchens/{kitchen_id}/recipes/{recipe_id}"

    ingredient_list = []
    
    print(ingredients)
    for ingredient in ingredients:
        ingredient_txt = ingredient['text'].lower().strip()
        if ingredient_txt not in bad_ingr and ingredient_txt:
            print(ingredient_txt)
            if ingredient['amount']['unit'] == 'g':
                ingred_dict = {
                    "id": ingredient_txt,
                    "names": [{"language": "de", "value": ingredient_txt}],
                    "amount": ingredient['amount']['quantity'],
                    "unit": "gram"}
                ingredient_list.append(ingred_dict)
        
    body = {
        "recipe": {
            "titles": [
                {
                    "language": "de",
                    "value": recipe_name
                }
            ],
            "date": "2020-09-19",
            "location": "Schweiz",
            "servings": 1,
            "ingredients": ingredient_list
        }
    }

    response = requests.put(url, json=body, auth=AUTH)
    
    if response.status_code in [601, 602]:
        print(response.text)
        bad_ingredient = re.findall(r"FoodService(.*)\\", response.text)[0].lower().strip()
        bad_ingr.append(bad_ingredient)
        print(bad_ingr)
        return put_recipe_migros(recipe_id, recipe_name, kitchen_id, ingredients, bad_ingr)
    
    elif response.status_code not in [200, 201, 202]:
        print(f"ERROR: Failed PUTting recipe {recipe_id} with status {response.status_code}: '{response.text}'")
    else:
        print(f"SUCCESS: PUT recipe {recipe_id}")
        print(response.json())
        print(bad_ingr)
        return response.json()


In [131]:
kitchen_id = "our_name_id"
create_kitchen("My First Kitchen", kitchen_id, "Switzerland")
bad_ingr = []
returns = []
for recipe in recipes:
    ingredients = recipe['ingredients']
    recipe_return = put_recipe_migros(recipe['id'], recipe['title'], kitchen_id, ingredients, bad_ingr)
    recipe_return['Veg'] = recipe['Veg']
    if recipe_return['Veg']:
        recipe_return['Vegan'] = recipe['Vegan']
    returns.append(recipe_return)

SUCCESS: PUT kitchen our_name_id
[{'id': '88766', 'is_main_ingredient': True, 'text': 'Schweinsbrust ohne Schwarte', 'amount': {'is_approximate': False, 'quantity': 300, 'quantity_up_to': None, 'text': '300 g', 'unit': 'g'}, 'availability': None, 'cultivar': None}, {'id': '16492', 'is_main_ingredient': False, 'text': 'grobkörniges Meersalz', 'amount': {'is_approximate': False, 'quantity': 1, 'quantity_up_to': None, 'text': '1 TL', 'unit': 'TL'}, 'availability': None, 'cultivar': None}, {'id': '16131', 'is_main_ingredient': False, 'text': 'Frühlingszwiebel', 'amount': {'is_approximate': False, 'quantity': 1, 'quantity_up_to': None, 'text': '1', 'unit': ''}, 'availability': None, 'cultivar': None}, {'id': '16499', 'is_main_ingredient': False, 'text': 'Ingwer', 'amount': {'is_approximate': False, 'quantity': 3, 'quantity_up_to': None, 'text': '3 cm', 'unit': 'cm'}, 'availability': None, 'cultivar': None}, {'id': '14769', 'is_main_ingredient': False, 'text': 'Wasser', 'amount': {'is_approx

SUCCESS: PUT recipe 88762
{'statuscode': 200, 'message': '', 'request-id': 0, 'recipe-id': '88762', 'recipe': {'id': '88762', 'kitchen-id': 'our_name_id', 'eaternity-award': False, 'rating': 'E', 'titles': [{'language': 'de', 'value': 'Geschmortes süss-salziges Schweinefleisch'}], 'date': '2020-09-19', 'location': 'Schweiz', 'servings': 1, 'recipe-portions': 1, 'ingredients': [], 'food-unit': 0.0}}
['schweinsbrust ohne schwarte']
[{'id': '88766', 'is_main_ingredient': True, 'text': 'Schweinsbrust ohne Schwarte', 'amount': {'is_approximate': False, 'quantity': 300, 'quantity_up_to': None, 'text': '300 g', 'unit': 'g'}, 'availability': None, 'cultivar': None}, {'id': '16492', 'is_main_ingredient': False, 'text': 'grobkörniges Meersalz', 'amount': {'is_approximate': False, 'quantity': 1, 'quantity_up_to': None, 'text': '1 TL', 'unit': 'TL'}, 'availability': None, 'cultivar': None}, {'id': '16131', 'is_main_ingredient': False, 'text': 'Frühlingszwiebel', 'amount': {'is_approximate': False,

SUCCESS: PUT recipe 88762
{'statuscode': 200, 'message': '', 'request-id': 0, 'recipe-id': '88762', 'recipe': {'id': '88762', 'kitchen-id': 'our_name_id', 'eaternity-award': False, 'rating': 'E', 'titles': [{'language': 'de', 'value': 'Geschmortes süss-salziges Schweinefleisch'}], 'date': '2020-09-19', 'location': 'Schweiz', 'servings': 1, 'recipe-portions': 1, 'ingredients': [], 'food-unit': 0.0}}
['schweinsbrust ohne schwarte']
[{'id': '88766', 'is_main_ingredient': True, 'text': 'Schweinsbrust ohne Schwarte', 'amount': {'is_approximate': False, 'quantity': 300, 'quantity_up_to': None, 'text': '300 g', 'unit': 'g'}, 'availability': None, 'cultivar': None}, {'id': '16492', 'is_main_ingredient': False, 'text': 'grobkörniges Meersalz', 'amount': {'is_approximate': False, 'quantity': 1, 'quantity_up_to': None, 'text': '1 TL', 'unit': 'TL'}, 'availability': None, 'cultivar': None}, {'id': '16131', 'is_main_ingredient': False, 'text': 'Frühlingszwiebel', 'amount': {'is_approximate': False,

SUCCESS: PUT recipe 88762
{'statuscode': 200, 'message': '', 'request-id': 0, 'recipe-id': '88762', 'recipe': {'id': '88762', 'kitchen-id': 'our_name_id', 'eaternity-award': False, 'rating': 'E', 'titles': [{'language': 'de', 'value': 'Geschmortes süss-salziges Schweinefleisch'}], 'date': '2020-09-19', 'location': 'Schweiz', 'servings': 1, 'recipe-portions': 1, 'ingredients': [], 'food-unit': 0.0}}
['schweinsbrust ohne schwarte']
[{'id': '88766', 'is_main_ingredient': True, 'text': 'Schweinsbrust ohne Schwarte', 'amount': {'is_approximate': False, 'quantity': 300, 'quantity_up_to': None, 'text': '300 g', 'unit': 'g'}, 'availability': None, 'cultivar': None}, {'id': '16492', 'is_main_ingredient': False, 'text': 'grobkörniges Meersalz', 'amount': {'is_approximate': False, 'quantity': 1, 'quantity_up_to': None, 'text': '1 TL', 'unit': 'TL'}, 'availability': None, 'cultivar': None}, {'id': '16131', 'is_main_ingredient': False, 'text': 'Frühlingszwiebel', 'amount': {'is_approximate': False,

SUCCESS: PUT recipe 88762
{'statuscode': 200, 'message': '', 'request-id': 0, 'recipe-id': '88762', 'recipe': {'id': '88762', 'kitchen-id': 'our_name_id', 'eaternity-award': False, 'rating': 'E', 'titles': [{'language': 'de', 'value': 'Geschmortes süss-salziges Schweinefleisch'}], 'date': '2020-09-19', 'location': 'Schweiz', 'servings': 1, 'recipe-portions': 1, 'ingredients': [], 'food-unit': 0.0}}
['schweinsbrust ohne schwarte']
[{'id': '88766', 'is_main_ingredient': True, 'text': 'Schweinsbrust ohne Schwarte', 'amount': {'is_approximate': False, 'quantity': 300, 'quantity_up_to': None, 'text': '300 g', 'unit': 'g'}, 'availability': None, 'cultivar': None}, {'id': '16492', 'is_main_ingredient': False, 'text': 'grobkörniges Meersalz', 'amount': {'is_approximate': False, 'quantity': 1, 'quantity_up_to': None, 'text': '1 TL', 'unit': 'TL'}, 'availability': None, 'cultivar': None}, {'id': '16131', 'is_main_ingredient': False, 'text': 'Frühlingszwiebel', 'amount': {'is_approximate': False,

SUCCESS: PUT recipe 88762
{'statuscode': 200, 'message': '', 'request-id': 0, 'recipe-id': '88762', 'recipe': {'id': '88762', 'kitchen-id': 'our_name_id', 'eaternity-award': False, 'rating': 'E', 'titles': [{'language': 'de', 'value': 'Geschmortes süss-salziges Schweinefleisch'}], 'date': '2020-09-19', 'location': 'Schweiz', 'servings': 1, 'recipe-portions': 1, 'ingredients': [], 'food-unit': 0.0}}
['schweinsbrust ohne schwarte']
[{'id': '88766', 'is_main_ingredient': True, 'text': 'Schweinsbrust ohne Schwarte', 'amount': {'is_approximate': False, 'quantity': 300, 'quantity_up_to': None, 'text': '300 g', 'unit': 'g'}, 'availability': None, 'cultivar': None}, {'id': '16492', 'is_main_ingredient': False, 'text': 'grobkörniges Meersalz', 'amount': {'is_approximate': False, 'quantity': 1, 'quantity_up_to': None, 'text': '1 TL', 'unit': 'TL'}, 'availability': None, 'cultivar': None}, {'id': '16131', 'is_main_ingredient': False, 'text': 'Frühlingszwiebel', 'amount': {'is_approximate': False,

SUCCESS: PUT recipe 88762
{'statuscode': 200, 'message': '', 'request-id': 0, 'recipe-id': '88762', 'recipe': {'id': '88762', 'kitchen-id': 'our_name_id', 'eaternity-award': False, 'rating': 'E', 'titles': [{'language': 'de', 'value': 'Geschmortes süss-salziges Schweinefleisch'}], 'date': '2020-09-19', 'location': 'Schweiz', 'servings': 1, 'recipe-portions': 1, 'ingredients': [], 'food-unit': 0.0}}
['schweinsbrust ohne schwarte']
[{'id': '88766', 'is_main_ingredient': True, 'text': 'Schweinsbrust ohne Schwarte', 'amount': {'is_approximate': False, 'quantity': 300, 'quantity_up_to': None, 'text': '300 g', 'unit': 'g'}, 'availability': None, 'cultivar': None}, {'id': '16492', 'is_main_ingredient': False, 'text': 'grobkörniges Meersalz', 'amount': {'is_approximate': False, 'quantity': 1, 'quantity_up_to': None, 'text': '1 TL', 'unit': 'TL'}, 'availability': None, 'cultivar': None}, {'id': '16131', 'is_main_ingredient': False, 'text': 'Frühlingszwiebel', 'amount': {'is_approximate': False,

KeyboardInterrupt: 

In [ ]:
for recipe in recipes:
    ingredients